해당 페이지에서는 직원, 프로그램, 프로젝트, 총 3가지 데이터를 통합하여 볼 수 있는 테이블 등을 생성하는데에 목적을 둔다

# Settings
#### Import Libraries

In [2]:
import os
import numpy as np
import pandas as pd

#### Set pandas option
table display 할 때에 컬럼이 많아도 전부 출력되도록 함

In [3]:
pd.set_option('display.max_rows', 5, 'display.max_columns', None)

# Import Data

In [4]:
root = os.path.join(os.getcwd(), 'DATA')

### 직원 데이터
행내 직원 기본 정보

In [5]:
# 직원
jikwon_dir = os.path.join(root, 'jikwon.csv')
jikwon = pd.read_csv(jikwon_dir, encoding='cp949')

### 프로그램 변경 이력 데이터
2020년 이후 신규/변경된 프로그램의 변경 이력 정보

용량 문제로 인하여 2개의 파일로 존재

In [6]:
# 프로그램 변경 이력
change_dir_1 = os.path.join(root, 'program_change_1.xlsx')
change_dir_2 = os.path.join(root, 'program_change_2.xlsx')
program_change_1 = pd.read_excel(change_dir_1)
program_change_2 = pd.read_excel(change_dir_2)

###  프로젝트 데이터
2020년 이후 프로그램을 신규/변경한 프로젝트 정보

In [7]:
# 프로젝트
project_dir = os.path.join(root, 'project.xlsx')
project = pd.read_excel(project_dir, sheet_name='프로젝트정보')
project_jikwon = pd.read_excel(project_dir, sheet_name='프로젝트인력')

---
# Preprocessing _ Jikwon

In [8]:
display(jikwon)

,JIKWON_NO,NAME,JEOM_NO,JEOM_NAME,JIKGUN,JIKGEUB,JIKWHI,JIKWHI_NAME,JUMINNO,JIKWHI2,JUJKMU_C,JUJKMU_NM,JUJKMU_RATE,BUJKMU_C,BUJKMU_NM,BUJKMU_RATE,JUMJANG_G,HOBONG,TOIJIKGBN,BUIM_ILJA,LST_CHG_ILSI,SILNO_CID,CUSNO,SAME_MN_NO
0,20505190,최태호,4,준법감시부,30000,99999,20425,전문계약직원,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,0,99998,0,20200401,NaN,fLOjQav3YWgEC,464464292,464464292
1,6209769,김명희,1107,영등포,20004,3,31053,부지점장겸 WRM,NaN,2,10905.0,WRM,100.0,NaN,NaN,0.0,0,10975,0,20190128,2.019013e+13,yoV049wbj4xLo,478015769,478015769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13971,20100700,이재상,1207,종로,20004,5,20258,행원,NaN,3,10612.0,상담 FC,90.0,10611.0,후선업무 AS,10.0,0,47345,0,20200206,2.020022e+13,l5xVJSARN5L8m,499646799,499646799
13972,6211895,김정아,1857,안양비산동,20004,4,20112,차장,NaN,2,10733.0,WM,100.0,NaN,NaN,0.0,0,23170,0,20200708,2.021021e+13,Ydg/fYYORY+c0,438419556,438419556


### 1. 점번호 필터링
데이터 대상자는 ICT 그룹 직원이므로 점번호를 기준으로 필터링

- 61(ICT운영부)
- 62(정보개발부)
- 63(금융개발부)
- 64(글로벌개발부)
- 66(기관개발부)
- 69(디지털개발부)
- 507(ICT기획부)

In [9]:
jikwon_data = jikwon[(jikwon['JEOM_NO'] == 61) | (jikwon['JEOM_NO'] == 62) | (jikwon['JEOM_NO'] ==  63) |
                     (jikwon['JEOM_NO'] == 64) | (jikwon['JEOM_NO'] == 66) | (jikwon['JEOM_NO'] == 69) | (jikwon['JEOM_NO'] ==  507)]

In [10]:
jikwon_data.shape

(432, 24)

432명의 ICT 그룹 직원이 있음을 확인할 수 있다

### 2. 코드 테이블 생성
문자열이나 코드로 이루어져 있거나 기타 이유로 사용할 수 없는 데이터가 있다면 제외해준다

**점정보**

In [11]:
jeom = jikwon_data[['JEOM_NO', 'JEOM_NAME']].drop_duplicates().reset_index(drop=True)

In [12]:
jeom

,JEOM_NO,JEOM_NAME
0,63,금융개발부
1,66,기관개발부
...,...,...
6,61,ICT운영부
7,507,정보개발부


**직위**

In [13]:
jikwhi = jikwon_data[['JIKWHI', 'JIKWHI_NAME']].drop_duplicates().reset_index(drop=True)

In [14]:
jikwhi

,JIKWHI,JIKWHI_NAME
0,20164,대리
1,20112,차장
...,...,...
9,20019,본부장
10,20399,팀장(부서장대우)


**직무**

In [15]:
jkmu = jikwon_data[['JUJKMU_C', 'JUJKMU_NM']].drop_duplicates().reset_index(drop=True)
jkmu.shape

(67, 2)

In [16]:
jkmu2 = jikwon_data[['BUJKMU_C', 'BUJKMU_NM']].drop_duplicates().reset_index(drop=True).rename(columns = {'BUJKMU_C': 'JUJKMU_C', 'BUJKMU_NM': 'JUJKMU_NM'}, inplace = False)
jkmu2.shape

(8, 2)

In [17]:
jkmu = pd.concat([jkmu, jkmu2], axis=0).drop_duplicates().reset_index(drop=True)

In [18]:
jkmu

,JUJKMU_C,JUJKMU_NM
0,11609.0,펌/가상
1,11614.0,시도금고 공금예금
...,...,...
66,10961.0,본부운용지원(기타)
67,11618.0,공공기관


### 3. 데이터 선별

In [19]:
jikwon_columns = ['JIKWON_NO', 'JEOM_NO', 'JIKGUN', 'JIKGEUB', 'JIKWHI', 'JIKWHI2', 'JUJKMU_C',
                  'JUJKMU_RATE', 'BUJKMU_C', 'BUJKMU_RATE', 'JUMJANG_G', 'HOBONG', 'BUIM_ILJA']

In [20]:
jikwon_data = jikwon_data[jikwon_columns].reset_index(drop=True)

### 4. 최종

In [21]:
display(jikwon_data)

,JIKWON_NO,JEOM_NO,JIKGUN,JIKGEUB,JIKWHI,JIKWHI2,JUJKMU_C,JUJKMU_RATE,BUJKMU_C,BUJKMU_RATE,JUMJANG_G,HOBONG,BUIM_ILJA
0,20101491,63,20004,5,20164,3,11609.0,100.0,NaN,0.0,0,43440,20210201
1,7101368,66,20004,4,20112,2,11614.0,100.0,NaN,0.0,0,26530,20180619
...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,10618082,61,40002,99999,20456,9,10961.0,100.0,NaN,0.0,0,99998,20200203
431,18101798,63,20004,5,20258,3,11029.0,100.0,NaN,0.0,0,45405,20180907


# Preprocessing _ Program

In [22]:
display(program_change_1)

,COL01,COL02,COL03,COL04,COL05,COL06,COL07,COL08,COL09,COL10,COL11,COL12,COL13,COL14,COL15,dw_lst_jukja_dt
0,코어,수신,수신 공통,채널매핑룰,A0_0003200002_SSDM0320A001_I,이창건,BK08101329,P2020-15765-01,20201228,수정,2020-04-059662,/webdev/chl1/CHLDownload/trans/mappingrule,코어,6,0,20210407
1,프로프레임4.0,JPK-일본ROC,글로벌여신,프로프레임소스(.c),sgle9999a.c,박창언,BK10101011,P2020-09188-02,20200818,신규,2020-04-042659,/jpsdev/gln/src,DS_글로벌뱅킹3,1,695,20210407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319996,프로프레임4.0,TSM-신신탁,신탁펀드등록,HTML5_xml,284016P002.xml,이세연,BK19100736,P2018-7039-01,20200109,수정,2020-04-002825,/src/tpm/tfr/screen/popup,신탁,3,4,20210407
319997,단위,(U2L)단위-통합CRM,단위-통합CRM,FLEX(/src 파일),F358135P000.mxml,신현국,BK06142869,P2020-03458-01,20200717,신규,2020-04-SCM731,/flexdev/src/ocs/ucs/screen/component/popup,ICTRM,1,326,20210407


### 1. Combine Program Data

데이터 조인에 앞서 프로그램 변경이력의 경우 2개의 dataframe으로 나뉘어져 있기 때문에 이를 합치도록 한다

In [23]:
program = pd.concat([program_change_1, program_change_2], axis=0)

In [24]:
print(program_change_1.shape)
print(program_change_2.shape)
print(program.shape)

(319998, 16)
(318125, 16)
(638123, 16)


### 2. 데이터 선별

In [25]:
program_columns = ['COL01', 'COL02', 'COL03', 'COL04', 'COL05', 'COL07', 'COL08', 'COL09', 'COL10', 'COL11', 'COL12']

In [26]:
program_data = program[program_columns]

### 3. 행번 전처리
직원 데이터의 직원번호는 **int형**인 반면, 프로그램 데이터의 직원번호는 'BK' + 행번 형태의 **object형**임을 확인할 수 있다

In [27]:
jikwon_data['JIKWON_NO']

0      20101491
1       7101368
         ...   
430    10618082
431    18101798
Name: JIKWON_NO, Length: 432, dtype: int64

In [28]:
program_data['COL07']

0         BK08101329
1         BK10101011
             ...    
318123    BK20200821
318124    BK16111008
Name: COL07, Length: 638123, dtype: object

프로그램 데이터의 직원번호를 직원데이터와 형태가 일치하도록 수정한다

그 전에, 모든 직원번호가 **BK + 행번** 형태인지 확인하자

In [29]:
program_data[program_data['COL07'].str[:2] != 'BK'].COL07

619       SGO001348
716       SGO001338
            ...    
317327    SGO001348
317900    SGO001354
Name: COL07, Length: 1107, dtype: object

이렇듯, BK로 시작하지 않는, 다시 말해 당행 직원이 아닌 데이터가 있다

해당 데이터는 현재 단계에서 필요하지 않기 때문에 제외하도록 한다

In [30]:
print(program_data.shape, end="->")
program_data = program_data[program_data['COL07'].str[:2] == 'BK']
print(program_data.shape)

(638123, 11)->(637016, 11)


이제 다시 직원데이터와 형태가 일치하도록 변경한다

In [31]:
program_data['COL07'] = program_data['COL07'].str[2:].astype(int)

### 3. Rename columns
Program 데이터의 경우 컬럼명이 실제 컬럼값을 대변하지 않는 것을 확인할 수 있는데, 컬럼값을 보면서 컬럼을 유추해야한다

In [32]:
rename_columns = {'COL01': '시스템구분', 'COL04': '프로그램종류', 'COL05': '프로그램명',
                  'COL07': '변경자', 'COL08': '프로젝트번호', 'COL09': '변경일자', 'COL10': '변경구분',
                  'COL11': '신청번호', 'COL12': '경로'}

In [33]:
program_data = program_data.rename(columns = rename_columns, inplace = False)

### 4. 날짜 데이터타입 변경
변경일자 컬럼은 날짜임에도 int형으로 되어있는데, 이를 날짜 형태로 바꾸도록 한다

In [34]:
program_data[['변경일자']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637016 entries, 0 to 318124
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   변경일자    637016 non-null  int64
dtypes: int64(1)
memory usage: 9.7 MB


In [35]:
program_data[['변경일자']] = pd.to_datetime(program_data['변경일자'], format='%Y%m%d')

In [36]:
program_data[['변경일자']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637016 entries, 0 to 318124
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   변경일자    637016 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 9.7 MB


### 5. 최종

In [37]:
display(program_data)

,시스템구분,COL02,COL03,프로그램종류,프로그램명,변경자,프로젝트번호,변경일자,변경구분,신청번호,경로
0,코어,수신,수신 공통,채널매핑룰,A0_0003200002_SSDM0320A001_I,8101329,P2020-15765-01,2020-12-28,수정,2020-04-059662,/webdev/chl1/CHLDownload/trans/mappingrule
1,프로프레임4.0,JPK-일본ROC,글로벌여신,프로프레임소스(.c),sgle9999a.c,10101011,P2020-09188-02,2020-08-18,신규,2020-04-042659,/jpsdev/gln/src
...,...,...,...,...,...,...,...,...,...,...,...
318123,프로프레임4.0,CAS-캐나다ROC,글로벌공통,FLEX(/src 파일),F429675A000as.as,20200821,P2020-13019-01,2020-12-14,수정,2020-04-057678,/flexdev/src/cas/gcm/screen
318124,웹컨텐츠,sharedPlatform2018,sharedPlatform2018,웹_JSP,NPIVST05020101_pilot.jsp,16111008,P2020-0818-01,2020-02-04,신규,2020-04-008829,/sharedPlatform2018/newPension/common/main


# Merge Data
이제 기본 인덱스로 사용할 직원 데이터에 다른 데이터를 조인하자

In [38]:
print(*jikwon_data.columns)

JIKWON_NO JEOM_NO JIKGUN JIKGEUB JIKWHI JIKWHI2 JUJKMU_C JUJKMU_RATE BUJKMU_C BUJKMU_RATE JUMJANG_G HOBONG BUIM_ILJA


In [39]:
print(*program_data.columns)

시스템구분 COL02 COL03 프로그램종류 프로그램명 변경자 프로젝트번호 변경일자 변경구분 신청번호 경로


## 직원 개인
직원 개개인이 어떤 프로젝트에 참여하여 어떤 프로그램을 신규/변경했는지 파악할 수 있는 테이블

In [40]:
raw_data = pd.merge(left=jikwon_data, right=program_data,
                     how='left', left_on='JIKWON_NO', right_on='변경자')
raw_data.drop(columns=['변경자'], inplace=True)

In [41]:
raw_data.describe()

,JIKWON_NO,JEOM_NO,JIKGUN,JIKGEUB,JIKWHI,JIKWHI2,JUJKMU_C,JUJKMU_RATE,BUJKMU_C,BUJKMU_RATE,JUMJANG_G,HOBONG,BUIM_ILJA
count,3.700400e+05,370040.000000,370040.000000,370040.000000,370040.000000,370040.000000,368745.00000,368745.00000,5340.000000,368745.00000,370040.000000,370040.000000,3.700400e+05
mean,9.779173e+06,96.993547,20052.898352,85.817655,20156.793395,2.311391,11030.12436,99.28056,11541.181461,0.71944,0.000051,25976.343574,2.018147e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75%,1.610146e+07,69.000000,20004.000000,5.000000,20164.000000,3.000000,11201.00000,100.00000,11605.000000,0.00000,0.000000,42120.000000,2.021020e+07
max,2.150232e+07,507.000000,40002.000000,99999.000000,31036.000000,9.000000,11641.00000,100.00000,11618.000000,50.00000,3.000000,99998.000000,2.021041e+07


# Preprocessing _ Project
프로젝트 인력 데이터부터 살펴보자

In [42]:
display(project_jikwon)

,프로젝트번호,행번,성명,부서명,팀명,직위,직급,역할,계획공수,실제공수
0,P2018-1020-01,16031001,김현주,글로벌개발부,DS_글로벌뱅킹1,S부부장,기타,PM,1829,2577.0
1,P2018-1020-01,06172202,손현수,금융개발부,IB,수석,Ma,개발자,1,1.0
...,...,...,...,...,...,...,...,...,...,...
29989,P2021-04489-01,19200854,박지현,정보개발부,Risk,선임,5급,PM,0,30.0
29990,P2021-04503-01,19501668,안지성,개발 Cell (SBJ DNX),개발 Cell (SBJ DNX),매니저,기타,PM,1,1.0


### 1. 인력 데이터 선별

In [43]:
project_jikwon_columns = ['프로젝트번호', '행번', '역할']

In [44]:
project_jikwon_data = project_jikwon[project_jikwon_columns]

다음, 프로젝트 데이터를 살펴보자

In [45]:
display(project)

,프로젝트번호,프로젝트명,프로젝트 상태,PM행번,PM성명,PM부서,업무명,프로그램변경유무,프로젝트계획시작일,프로젝트계획종료일,착수일,종료일,이행예정일,이행일,개발의뢰번호,의뢰자행번,의뢰자정보,품질검증대상유무,패스트체크인여부,체크인가능유무,인수테스트 완료여부,BA결재 여부,인수테스트대상유무,인수테스트 실시건수,인수테스트 마지막 등록일,통합테스트유무,Helpdesc공지대상유무,Helpdesc공지대상구분
0,P2018-1020-01,IBMS 지분투자 플랫폼 개선,종료,16031001,김현주,DS_종금신탁,IB 프로젝트 기획관리,Y,20180206,20200930,20180206.0,20200928.0,20200930,20200921,R2018-1020,6228623,박영애 기타(투자금융부),N,N,Y,Y,N,Y,1,20190121.0,N,N,NaN
1,P2018-6996-01,[캄보디아] CSS 연동을 통한 CD공동망 개발,종료,6127258,고규범,글로벌수신,채널-Dex,Y,20180920,20200214,20180920.0,20200205.0,20200318,20200203,R2018-6996,6167675,김춘호 기타(본점),N,Y,Y,Y,N,Y,1,20200130.0,Y,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11902,P2021-04489-01,리스크업무 자체개선(4월),종료,19200854,박지현,Risk,신용리스크,Y,20210405,20210507,20210405.0,20210507.0,20210507,20210506,R2021-04489,19200854,박지현 선임(Risk),N,Y,Y,N,N,N,0,NaN,N,Y,기타
11903,P2021-04503-01,[SBJ] UMS 이미지 신규 등록,종료,19501668,안지성,개발 Cell (SBJ DNX),SBJ은행,Y,20210405,20210406,20210405.0,20210408.0,20210406,20210405,R2021-04503,19501668,안지성 기타(개발 Cell (SBJ DNX)),N,Y,Y,Y,N,Y,1,20210405.0,N,N,NaN


### 2. 프로젝트 데이터 선별

In [46]:
project_columns = ['프로젝트번호', '업무명',
                   '프로젝트계획시작일', '프로젝트계획종료일', '착수일', '종료일', '이행예정일', '이행일',
                   '개발의뢰번호', '의뢰자행번']

In [47]:
project_data = project[project_columns]

### 3. 날짜 전처리
프로젝트 관련 날짜는 '프로젝트계획시작일', '프로젝트계획종료일', '착수일', '종료일', '이행예정일', '이행일', 총 6종류가 있다

우선 해당 컬럼의 데이터 형태를 datetime으로 바꿔주자

In [48]:
date_columns = ['프로젝트계획시작일', '프로젝트계획종료일', '착수일', '종료일', '이행예정일', '이행일']

In [49]:
project_data[date_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11904 entries, 0 to 11903
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   프로젝트계획시작일  11904 non-null  int64  
 1   프로젝트계획종료일  11904 non-null  int64  
 2   착수일        11902 non-null  float64
 3   종료일        11705 non-null  float64
 4   이행예정일      11904 non-null  int64  
 5   이행일        11904 non-null  int64  
dtypes: float64(2), int64(4)
memory usage: 558.1 KB


In [50]:
project_data[date_columns] = project_data[date_columns].apply(pd.to_datetime,format='%Y%m%d', errors='coerce')

C:\Users\13579\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [51]:
project_data[date_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11904 entries, 0 to 11903
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   프로젝트계획시작일  11904 non-null  datetime64[ns]
 1   프로젝트계획종료일  11904 non-null  datetime64[ns]
 2   착수일        11902 non-null  datetime64[ns]
 3   종료일        11705 non-null  datetime64[ns]
 4   이행예정일      11904 non-null  datetime64[ns]
 5   이행일        11904 non-null  datetime64[ns]
dtypes: datetime64[ns](6)
memory usage: 558.1 KB


프로젝트 실행 기간을 알기 위해 이를 조정하여 최초 시행일자, 최종 실행일자, 그리고 기간, 총 3가지 컬럼을 만들어보자

In [52]:
project_data['최초일자'] = project_data[['프로젝트계획시작일', '착수일']].min(axis=1)
project_data['최종일자'] = project_data[['프로젝트계획종료일', '종료일', '이행예정일', '이행일']].max(axis=1)
project_data['기간'] = project_data['최종일자'] - project_data['최초일자']

<ipython-input-52-a90a681c45b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  project_data['최초일자'] = project_data[['프로젝트계획시작일', '착수일']].min(axis=1)
<ipython-input-52-a90a681c45b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  project_data['최종일자'] = project_data[['프로젝트계획종료일', '종료일', '이행예정일', '이행일']].max(axis=1)
<ipython-input-52-a90a681c45b9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [53]:
project_data.drop(columns=['프로젝트계획시작일', '프로젝트계획종료일', '착수일', '종료일', '이행예정일', '이행일'], inplace=True)

C:\Users\13579\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### 4. 인력정보에 프로젝트 정보 추가

In [54]:
project_data = pd.merge(left=project_jikwon_data, right=project_data,
                       how='left', left_on='프로젝트번호', right_on='프로젝트번호')

### 5. 최종

In [55]:
display(project_data)

,프로젝트번호,행번,역할,업무명,개발의뢰번호,의뢰자행번,최초일자,최종일자,기간
0,P2018-1020-01,16031001,PM,IB 프로젝트 기획관리,R2018-1020,6228623,2018-02-06,2020-09-30,967 days
1,P2018-1020-01,06172202,개발자,IB 프로젝트 기획관리,R2018-1020,6228623,2018-02-06,2020-09-30,967 days
...,...,...,...,...,...,...,...,...,...
29989,P2021-04489-01,19200854,PM,신용리스크,R2021-04489,19200854,2021-04-05,2021-05-07,32 days
29990,P2021-04503-01,19501668,PM,SBJ은행,R2021-04503,19501668,2021-04-05,2021-04-08,3 days


# Export Data
데이터 전처리를 완료하였으니, 해당 Dataframe을 csv 파일로 저장한다

In [56]:
raw_data.to_csv(os.path.join(root, 'raw_data.csv'), na_rep='NaN')
project_data.to_csv(os.path.join(root, 'project_data.csv'), na_rep='NaN')
jeom.to_csv(os.path.join(root, 'jeom_code.csv'), na_rep='NaN')
jikwhi.to_csv(os.path.join(root, 'jikwhi_code.csv'), na_rep='NaN')
jkmu.to_csv(os.path.join(root, 'jkmu_code.csv'), na_rep='NaN')